In [1]:
import os
import sys
import jsonlines
from loguru import logger
from pathlib import Path
home_path = Path().absolute().parents[1]
logger.info(f"home path: {home_path.as_posix()}")

logger.info('设置推理卡片为1,2')
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
sys.path.append(home_path.as_posix())

from ultrarag.modules.llm import OpenaiLLM
from ultrarag.modules.database import QdrantIndex
from ultrarag.modules.embedding import MiniCPMEmbServer


prompt = "请结合召回结果回答用户问题：\n召回结果：\n{recalls}\n用户问题：\n{query}\n回答："

corpus_path = home_path / 'examples' / 'workflow' / 'corpus.jsonl'
data = list(jsonlines.open(corpus_path.as_posix(), 'r'))


embedding_model_path = "/mnt1/guodewen/models/MiniCPM-Embedding-Light"
qdrant_collection_path = (home_path / 'resource' / 'qdrant').as_posix()

2025-04-21 20:45:44.099 | INFO     | __main__:<module>:7 - home path: /mnt1/guodewen/research/UltraRAG
2025-04-21 20:45:44.099 | INFO     | __main__:<module>:9 - 设置推理卡片为1,2


INFO 04-21 20:45:47 __init__.py:183] Automatically detected platform cuda.


/mnt1/guodewen/research/UltraRAG/ultrarag/modules/database/__init__.py:8: UserWarning: failed to load MilvusIndex beacause not available qdrant_client, ignored it if you do not need it
  warnings.warn("failed to load MilvusIndex beacause not available qdrant_client, ignored it if you do not need it")


#### 初始化函数说明

> OpenaiLLM(model, base_url, api_key, **kargs) -> client
> 初始化推理模型
*入参*
- model: 默认的模型名称，如果请求时不指定则使用这个 model 名称推理
- base_url: 推理接口地址
- api_key: 推理接口密钥
- kargs: 字典结构，作为模型请求时的其他默认参数，如温度系数、topk, top-p等
*输出*
- client：输出一个 object

> MiniCPMEmbServer(url_or_path, batch_size, max_length, query_instruction, document_instruction, **kargs)
> 初始化文本向量化模型

**入参**
- url_or_path: 加载推理模型时指定模型的路径
- batch_size: 批量请求时限制推理的并发，避免显存 OOM
- max_length: 推理请求时限制文本的长度，避免显存 OOM
- query_instruction: 对于有些模型，需要在 encode 之前拼上特殊指令，例如 bge large
- document_instruction: 对于有些模型，需要在 encode 之前拼上特殊指令，例如 bge large

**输出**
- client：输出一个 object

> QdrantIndex(url, encoder)
> 初始化数据库

**入参**
- url: 加载知识库时指定数据库的路径
- encoder: 对于向量数据库，需要指定推理模型将文本映射成向量，对于全文检索数据库，可以为空

In [2]:
llm = OpenaiLLM(
    model="QwQ-32B",
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
)

embed = MiniCPMEmbServer(url_or_path=embedding_model_path)

index = QdrantIndex(url=qdrant_collection_path, encoder=embed)

2025-04-21 20:45:48.363 | INFO     | ultrarag.modules.llm.openai_like:__init__:19 - api_key: EMPTY, base_url: http://localhost:8000/v1, kargs: {'model': 'QwQ-32B'}
2025-04-21 20:45:49.911 | INFO     | ultrarag.modules.embedding.minicpm_embedding:load_model:145 - The parameters of colbert_linear and sparse linear is new initialize. Make sure the model is loaded for training, not inferencing


----------using 2*GPUs----------


#### 构建知识库说明

> index.get_collections()
> 获取知识库列表

> index.create(collection_name, dimension):
> 创建新的索引
- collection_name: 创建索引的名称
- dimension: 对于向量数据库，需要指定初始化向量的维度

> index.insert(collection, payloads, func, callback):
> 插入数据库数据

- collection: 待插入目标索引的名称
- payloads: 待插入数据，是一个 list,每个元素是一个字典，或者说是一个 json [TODO:后面需要考虑大文件的情况，通过传入文件生成器减少内存占用]
- func: 这个函数是一个回调函数，用来告诉数据库对哪些信息建索引；对于向量数据库来说，就是确定 mebedding 文本的拼接规则
- callback: 对于大文件插入， 传输 callback 用于显示处理的进度

In [3]:
if "demo" not in index.get_collections():
    await index.create("demo")
    
await index.insert(collection="demo", payloads=data, func=lambda x: x["title"] + x["content"])

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### 检索生成

> index.search(collection, query, topn, method)
> 检索召回文档，由于检索耗时所以是异步调用

**入参**
- collection: 指定要检索的索引名称, 可以是一个或者是多个，按照得分排序取 topn
- query: 待检索的 query 
- topn：控制召回的候选数目
- method: 支持稠密检索和混合检索

**出参**
- return: 返回一个BaseNode 格式的列表，其中包含相似度得分、content 和原始的 payloads 信息

> llm.arun(messages, stream, **kargs)
> 生成回复

**入参**
- messages：对话流信息，格式和 openai 保持一致
- stream: 用于标记是否流式输出
- kargs: 推理模型请求所需要的其他参数，例如指定模型名称、温度值等等

**出参**
- return: 返回一个输出的字符串（非流式）,或者一个异步生成器(流式)

In [4]:
query="宫保鸡丁的做法"

results = await index.search(collection="demo", query=query, topn=3)
logger.info(f"召回结果：\n{results}")

message = [
    dict(role="user", content=prompt.format(query=query, recalls=results)),
]
response = await llm.arun(messages=message, stream=False)
logger.info(f"推理结果：\n{response}")

2025-04-21 20:45:51.672 | INFO     | __main__:<module>:4 - 召回结果：
[BaseNode(score=0.6624137391062277, content='9. **宫保鸡丁**\\n\\n**食材准备**：鸡胸肉250g、花生米一小碗、干辣椒10个、花椒适量、葱姜蒜各适量、生抽、老抽、醋、糖、淀粉。\\n\\n**做法详解**：  \\n鸡胸肉洗净切成1厘米见方的小丁，用料酒、生抽、淀粉腌制15分钟，帮助鸡肉保持嫩滑口感。干辣椒剪成小段，花生米提前用小火炒熟备用。调料汁：生抽1勺、老抽半勺、糖1勺、醋1.5勺、水适量、淀粉1勺，搅拌均匀成宫保汁。\\n\\n锅中加油烧热，下鸡丁快速翻炒至变色后盛出。锅中留底油，放入干辣椒和花椒，用小火炒出香味（不要糊），然后加入葱姜蒜炒香，接着倒入鸡丁快速翻炒。最后淋入宫保汁，翻炒至浓稠，再撒入炒熟的花生米，炒匀即可。\\n\\n**小贴士**：干辣椒先炒香但不能炒糊；宫保汁一次倒入，快速翻炒收汁；花生最后放入保持脆感。', payload={'title': '宫保鸡丁', 'content': '9. **宫保鸡丁**\\n\\n**食材准备**：鸡胸肉250g、花生米一小碗、干辣椒10个、花椒适量、葱姜蒜各适量、生抽、老抽、醋、糖、淀粉。\\n\\n**做法详解**：  \\n鸡胸肉洗净切成1厘米见方的小丁，用料酒、生抽、淀粉腌制15分钟，帮助鸡肉保持嫩滑口感。干辣椒剪成小段，花生米提前用小火炒熟备用。调料汁：生抽1勺、老抽半勺、糖1勺、醋1.5勺、水适量、淀粉1勺，搅拌均匀成宫保汁。\\n\\n锅中加油烧热，下鸡丁快速翻炒至变色后盛出。锅中留底油，放入干辣椒和花椒，用小火炒出香味（不要糊），然后加入葱姜蒜炒香，接着倒入鸡丁快速翻炒。最后淋入宫保汁，翻炒至浓稠，再撒入炒熟的花生米，炒匀即可。\\n\\n**小贴士**：干辣椒先炒香但不能炒糊；宫保汁一次倒入，快速翻炒收汁；花生最后放入保持脆感。'}), BaseNode(score=0.3389509456255837, content='2. **青椒土豆丝**\\n\\n**食材准备**：土豆2个、青椒1个、大蒜2瓣、白醋1勺、盐适量、食用油适量。\\n\\n**做法详解**：  \\n将土豆去皮，先切成薄片

In [5]:
# 删除知识库
await index.remove("demo")